In [0]:
import logging
import time,datetime

In [0]:
logging.debug("This is a debug message"); #types of logging information
logging.info("This is an info message");
logging.warning("This is a warning message");
logging.error("This is an error message");
logging.fatal("This is a critical error");

ERROR:root:This is an error message
CRITICAL:root:This is a critical error


In [0]:
logger=logging.getLogger("log4j")
logger.setLevel(logging.DEBUG)

 I used log4j logger and I am  setting up level as DEBUG as debug gives includes info and error messages we can alo also use  levels as info,error See if you use info level it  gives only info messages  ,error level  gives only error messages  so better use debug level and formatter will help to store the file name as in the particular format

In [0]:
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(funcName)s: %(message)s',
    datefmt='%d/%m/%Y %I:%M:%S %p'
)  #first formatter
     


In [0]:
formatter = logging.Formatter(
    '%(levelname)s (%(asctime)s): %(message)s (Line: %(lineno)d [%(filename)s])',
    datefmt='%d/%m/%Y %I:%M:%S %p'
) #second formatter
     

 how your log file will be is decided by formatter and if data is from multiple timezone use python module to handle that

 databricks can access only dbfs mounted folder i.e mostly /dbfs folder it cannot access local folders .

In [0]:
import os 
os.makedirs("/tmp/logs/", exist_ok=True)
log_filename = f"/tmp/logs/{datetime.datetime.now().strftime('%Y-%m-%d-%H')}.log"

logFileHandler = logging.FileHandler(log_filename, mode='w')  # 'a' will append
logFileHandler.setFormatter(formatter)
logger.addHandler(logFileHandler)

 filehandler  stores the file in the particular folder and tell which format and add that logfile to the log4j logger

In [0]:
#from pyspark.sql import SparkSession
#spark=SparkSession.builder.appName('Dataframe').getOrCreate() #starting spark session # used if it is not databricks

In [0]:
dbutils.fs.ls('/FileStore/tables/')

Out[6]: [FileInfo(path='dbfs:/FileStore/tables/BigMart_Sales.csv', name='BigMart_Sales.csv', size=869537, modificationTime=1747908600000)]

In [0]:
df = (spark.read.format("csv") 
    .option("header", True) 
    .option("inferSchema", True) 
    .load('dbfs:/FileStore/tables/BigMart_Sales.csv') ) 


In [0]:
display(df.limit(10))


Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138
DRC01,5.92,Regular,0.019278216,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
FDN15,17.5,Low Fat,0.016760075,Meat,141.618,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.27
FDX07,19.2,Regular,0,Fruits and Vegetables,182.095,OUT010,1998,null,Tier 3,Grocery Store,732.38
NCD19,8.93,Low Fat,0,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
FDP36,10.395,Regular,0,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
FDO10,13.65,Regular,0.012741089,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
FDP10,null,Low Fat,0.127469857,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
FDH17,16.2,Regular,0.016687114,Frozen Foods,96.9726,OUT045,2002,null,Tier 2,Supermarket Type1,1076.5986
FDU28,19.2,Regular,0.09444959,Frozen Foods,187.8214,OUT017,2007,null,Tier 2,Supermarket Type1,4710.535


In [0]:
dbutils.widgets.text("workspace_name", "my_workspace", "Workspace Name")

In [0]:
dbutils.widgets.text("workspace_name", "", "Workspace Name")
workspace_name = dbutils.widgets.get("workspace_name")
print(workspace_name)

dev workspace


In [0]:
full_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
notebook_name = full_path.split("/")[-1]
print(notebook_name)

logging notebook


In [0]:
log_context = f"{workspace_name}/{notebook_name}"
print(f"Log context: {log_context}")


Log context: dev workspace/logging notebook


In [0]:
message = '{} Share data source file read started................'.format(log_context)
logger.info(message)

# Read file
df = spark.read.csv('/FileStore/tables/BigMart_Sales.csv', header='true')
count = df.count()
print(count)

message = '{} Total number of {} records present'.format(log_context, count)
logger.info(message)

# Try-except for error logging
try:
    df1 = df.withColumn('testColumn', str(df.column_not_present))  # Intentional error
    display(df1)

except Exception as e:
    print('Logging Error {}'.format(e))
    errorMessage = '{} Logging Error defined: {}'.format(log_context, e)
    logger.error(errorMessage)

INFO:log4j:dev workspace/logging notebook Share data source file read started................
INFO:log4j:dev workspace/logging notebook Total number of 8523 records present
ERROR:log4j:dev workspace/logging notebook Logging Error defined: 'DataFrame' object has no attribute 'column_not_present'


8523
Logging Error 'DataFrame' object has no attribute 'column_not_present'


In [0]:
logging.shutdown()

In [0]:
%sh
ls /tmp/logs

2025-05-26-16.log


In [0]:
dbutils.fs.cp("file:/tmp/logs/2025-05-26-16.log", "dbfs:/tmp/logs/2025-05-26-16.log")


Out[16]: True

In [0]:
de=spark.read.text('dbfs:/tmp/logs/2025-05-26-16.log') #create by first formatter
de.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                 |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|INFO (26/05/2025 04:07:12 PM): dev workspace/logging notebook Share data source file read started................ (Line: 2 [<command-2696206594891380>])                              |
|INFO (26/05/2025 04:07:16 PM): dev workspace/logging notebook Total number of 8523 records present (Line: 10 [<command-2696206594891380>])                                            |
|ERROR (26/05/2025 04:07:16 PM): dev workspace/logging notebook Logging Err